# Model Implementation

In [ ]:
import numpy as np
from collections import Counter

class TreeNode:
    def __init__(self, feature_index=None, threshold_value=None, left_subtree=None, right_subtree=None, *, leaf_value=None):
        """
        Represents a node in the decision tree.

        Parameters:
        feature_index (int): Index of the feature to split on.
        threshold_value (float): Threshold value for splitting the data.
        left_subtree (TreeNode): Left child node.
        right_subtree (TreeNode): Right child node.
        leaf_value (int/float): Value if the node is a leaf (i.e., the prediction).
        """
        self.feature_index = feature_index
        self.threshold_value = threshold_value
        self.left_subtree = left_subtree
        self.right_subtree = right_subtree
        self.leaf_value = leaf_value

    def is_leaf_node(self):
        """Check if the node is a leaf node."""
        return self.leaf_value is not None

class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_tree_depth=100, max_features=None):
        """
        Decision Tree Classifier.

        Parameters:
        min_samples_split (int): Minimum number of samples required to split a node.
        max_tree_depth (int): Maximum depth of the tree.
        max_features (int): Maximum number of features to consider when looking for the best split.
        """
        self.min_samples_split = min_samples_split
        self.max_tree_depth = max_tree_depth
        self.max_features = max_features
        self.root_node = None

    def fit(self, X, y):
        """
        Build the decision tree classifier from the training set (X, y).

        Parameters:
        X (array-like): Feature matrix of shape (n_samples, n_features).
        y (array-like): Target values of shape (n_samples,).
        """
        num_features = X.shape[1]
        # If max_features is not set, use all features
        self.max_features = num_features if not self.max_features else min(num_features, self.max_features)
        self.root_node = self._grow_tree(X, y)

    def _grow_tree(self, X, y, current_depth=0):
        """
        Recursively grow the decision tree.

        Parameters:
        X (array-like): Feature matrix of the current node.
        y (array-like): Target values of the current node.
        current_depth (int): Current depth of the tree.

        Returns:
        TreeNode: The root node of the grown tree.
        """
        num_samples, num_features = X.shape
        num_unique_labels = len(np.unique(y))

        # Check stopping criteria
        if (current_depth >= self.max_tree_depth or num_unique_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._calculate_most_common_label(y)
            return TreeNode(leaf_value=leaf_value)

        # Select a subset of features to consider for splitting
        feature_indices = np.random.choice(num_features, self.max_features, replace=False)

        # Find the best feature and threshold to split on
        best_feature_index, best_threshold_value = self._determine_best_split(X, y, feature_indices)

        # Split the dataset into left and right branches
        left_indices, right_indices = self._split_dataset(X[:, best_feature_index], best_threshold_value)
        left_subtree = self._grow_tree(X[left_indices, :], y[left_indices], current_depth + 1)
        right_subtree = self._grow_tree(X[right_indices, :], y[right_indices], current_depth + 1)

        # Return a new node with the best split
        return TreeNode(best_feature_index, best_threshold_value, left_subtree, right_subtree)

    def _determine_best_split(self, X, y, feature_indices):
        """
        Determine the best feature and threshold to split the data.

        Parameters:
        X (array-like): Feature matrix.
        y (array-like): Target values.
        feature_indices (array-like): Indices of features to consider.

        Returns:
        tuple: Index of the best feature and the best threshold value.
        """
        best_information_gain = -1  # Initialize the best information gain to be negative
        best_feature_index = None  # Initialize the best feature index
        best_threshold_value = None  # Initialize the best threshold value

        # Iterate over each feature index
        for feature_index in feature_indices:
            feature_values = X[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            # Iterate over each unique threshold value for the current feature
            for threshold_value in possible_thresholds:
                # Calculate information gain for this split
                information_gain = self._calculate_information_gain(y, feature_values, threshold_value)

                # Update best split if the current split is better
                if information_gain > best_information_gain:
                    best_information_gain = information_gain
                    best_feature_index = feature_index
                    best_threshold_value = threshold_value

        return best_feature_index, best_threshold_value

    def _calculate_information_gain(self, y, feature_values, threshold_value):
        """
        Calculate the information gain of a split.

        Parameters:
        y (array-like): Target values.
        feature_values (array-like): Feature values corresponding to the split.
        threshold_value (float): The threshold value to split on.

        Returns:
        float: The information gain from the split.
        """
        # Calculate parent entropy
        parent_entropy = self._calculate_entropy(y)

        # Split the data into left and right branches
        left_indices, right_indices = self._split_dataset(feature_values, threshold_value)

        # If no valid split, return zero information gain
        if len(left_indices) == 0 or len(right_indices) == 0:
            return 0

        # Calculate the weighted average entropy of the child nodes
        total_samples = len(y)
        left_weight = len(left_indices) / total_samples
        right_weight = len(right_indices) / total_samples
        child_entropy = (left_weight * self._calculate_entropy(y[left_indices]) +
                         right_weight * self._calculate_entropy(y[right_indices]))

        # Information gain is the difference between parent entropy and weighted child entropy
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split_dataset(self, feature_values, threshold_value):
        """
        Split the dataset based on the threshold value.

        Parameters:
        feature_values (array-like): Feature values corresponding to the split.
        threshold_value (float): The threshold value to split on.

        Returns:
        tuple: Indices of the left and right splits.
        """
        left_indices = np.argwhere(feature_values <= threshold_value).flatten()
        right_indices = np.argwhere(feature_values > threshold_value).flatten()
        return left_indices, right_indices

    def _calculate_entropy(self, y):
        """
        Calculate the entropy of a distribution.

        Parameters:
        y (array-like): Target values.

        Returns:
        float: Entropy of the distribution.
        """
        label_counts = np.bincount(y)
        probabilities = label_counts / len(y)
        # Entropy is the sum of -p * log(p) for each probability p
        return -np.sum([p * np.log(p) for p in probabilities if p > 0])

    def _calculate_most_common_label(self, y):
        """
        Determine the most common label in the target values.

        Parameters:
        y (array-like): Target values.

        Returns:
        int/float: The most common label.
        """
        label_counter = Counter(y)
        most_common_label = label_counter.most_common(1)[0][0]
        return most_common_label

    def predict(self, X):
        """
        Predict class labels for the input samples.

        Parameters:
        X (array-like): Feature matrix of shape (n_samples, n_features).

        Returns:
        array: Predicted class labels.
        """
        return np.array([self._traverse_tree(sample, self.root_node) for sample in X])

    def _traverse_tree(self, sample, node):
        """
        Traverse the decision tree to make a prediction for a single sample.

        Parameters:
        sample (array-like): A single data sample.
        node (TreeNode): The current node in the tree.

        Returns:
        int/float: The predicted class label.
        """
        # If the current node is a leaf, return its value (the prediction)
        if node.is_leaf_node():
            return node.leaf_value

        # Recurse down the tree based on the feature and threshold at the current node
        if sample[node.feature_index] <= node.threshold_value:
            return self._traverse_tree(sample, node.left_subtree)
        return self._traverse_tree(sample, node.right_subtree)


# Test model - Breast cancer dataset

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Load the breast cancer dataset
breast_cancer_data = datasets.load_breast_cancer()
X, y = breast_cancer_data.data, breast_cancer_data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree model
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = decision_tree_model.predict(X_test)

def calculate_accuracy(y_true, y_pred):
    """
    Calculate the accuracy of predictions.

    Parameters:
    y_true (array-like): True labels.
    y_pred (array-like): Predicted labels.

    Returns:
    float: Accuracy of the predictions.
    """
    return np.sum(y_true == y_pred) / len(y_true)

# Calculate and print the accuracy of the model
accuracy = calculate_accuracy(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 93.86%
